In [1]:
!ls ../wt103/bwd_wt103.h5

../wt103/bwd_wt103.h5


In [2]:
PRE_PATH = "data/aclImdb/models/wt103"

In [3]:
import torch
from pprint import pprint
weights = torch.load(PRE_PATH+'/fwd_wt103.h5',map_location=lambda storage, loc: storage)
pprint([(weight_key, weights[weight_key].size()) for weight_key in weights.keys()])

[('0.encoder.weight', torch.Size([238462, 400])),
 ('0.encoder_with_dropout.embed.weight', torch.Size([238462, 400])),
 ('0.rnns.0.module.weight_ih_l0', torch.Size([4600, 400])),
 ('0.rnns.0.module.bias_ih_l0', torch.Size([4600])),
 ('0.rnns.0.module.bias_hh_l0', torch.Size([4600])),
 ('0.rnns.0.module.weight_hh_l0_raw', torch.Size([4600, 1150])),
 ('0.rnns.1.module.weight_ih_l0', torch.Size([4600, 1150])),
 ('0.rnns.1.module.bias_ih_l0', torch.Size([4600])),
 ('0.rnns.1.module.bias_hh_l0', torch.Size([4600])),
 ('0.rnns.1.module.weight_hh_l0_raw', torch.Size([4600, 1150])),
 ('0.rnns.2.module.weight_ih_l0', torch.Size([1600, 1150])),
 ('0.rnns.2.module.bias_ih_l0', torch.Size([1600])),
 ('0.rnns.2.module.bias_hh_l0', torch.Size([1600])),
 ('0.rnns.2.module.weight_hh_l0_raw', torch.Size([1600, 400])),
 ('1.decoder.weight', torch.Size([238462, 400]))]


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
!pip install h5py

In [6]:
recast_weights = {}
for i in range(3):
    for op in ['weight','bias']:
        for segment in ['ih','hh']:
            source_key = "0.rnns.{0}.module.{1}_{2}_l0{3}"\
            .format(i,op,segment,"_raw" if op=='weight' and segment == 'hh' else "")
            target_key = "{0}.{1}_{2}_l0".format(i,op,segment)
            recast_weights[target_key] = weights[source_key]
recast_weights

{'0.weight_ih_l0': tensor([[-0.0812, -0.0811, -0.0937,  ..., -0.0259, -0.1403, -0.3247],
         [ 0.1154,  0.1142,  0.0938,  ..., -0.0711,  0.1669, -0.0387],
         [-0.0051,  0.1007,  0.2071,  ..., -0.0860, -0.0288, -0.0894],
         ...,
         [ 0.0055,  0.0157,  0.2990,  ...,  0.0616,  0.1159, -0.4737],
         [ 0.0181,  0.0426,  0.1130,  ...,  0.3529, -0.0114, -0.0125],
         [-0.0167, -0.1328,  0.1741,  ...,  0.0548, -0.0045,  0.1688]]),
 '0.weight_hh_l0': tensor([[-0.1013,  0.1786, -0.0528,  ...,  0.0741,  0.0306,  0.2467],
         [ 0.1780, -0.0853, -0.0243,  ..., -0.1129, -0.1310, -0.1498],
         [ 0.0661, -0.0496,  0.0921,  ...,  0.1829,  0.0533, -0.1525],
         ...,
         [-0.0322, -0.0704,  0.1653,  ...,  0.2142, -0.0558,  0.0315],
         [-0.1651, -0.0290,  0.1748,  ..., -0.0446,  0.5444,  0.0616],
         [ 0.0905, -0.1704, -0.0053,  ..., -0.0057,  0.2269,  0.0328]]),
 '0.bias_ih_l0': tensor([ 0.1503, -0.4701, -0.1885,  ..., -0.5919, -0.2172, -0.1

In [7]:
from torchtext import data
from torchtext.datasets import LanguageModelingDataset
import re

TEXT = data.Field(lower=True, batch_first=True, tokenize="spacy",
                  eos_token="<eos>",
                  pad_token="_pad_",
                  unk_token="_unk_")

# SOURCE = "war_and_peace.txt"
SOURCE = "../data/creepypasta_corpus.txt"
TMP = "tmp.txt"
with open(SOURCE,'r') as inp:
    with open(TMP,'w') as out:
        data = inp.read()
        data = re.sub("\n\s*\n?\s*","\n",data)
        data = re.sub("[“”]","\"",data)
        out.write(data)

dataset = LanguageModelingDataset(TMP,TEXT,newline_eos=False)[0].text

In [8]:
import pickle
with open((PRE_PATH+'/itos_wt103.pkl'),'rb') as f:
    itos2 = pickle.load(f)
print(len(itos2))
original_vocab_size = len(itos2)

from collections import Counter
itos2_set = set(itos2)
vocabs = Counter(dataset)
for word,_ in vocabs.most_common():
    if word not in itos2_set:
        itos2.append(word)
print(len(itos2))
new_vocab_size = len(itos2)

238462
257874


In [9]:
import collections

stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

In [10]:
vocabs.most_common()

[('.', 494949),
 ('the', 424604),
 (',', 399877),
 ('i', 247087),
 ('to', 206684),
 ('and', 197047),
 ('a', 161930),
 ('of', 153551),
 ('"', 146794),
 ('was', 140283),
 ('it', 123965),
 ('in', 106073),
 ('my', 95290),
 ('that', 93893),
 ('he', 82974),
 ('you', 63872),
 ('as', 59234),
 ('his', 56211),
 ('n’t', 55689),
 ('had', 55679),
 ('me', 53541),
 ('on', 51110),
 ('but', 49820),
 ('her', 48980),
 ('’s', 48872),
 ('with', 47942),
 ('for', 47238),
 ('she', 46851),
 ('at', 44376),
 ('this', 36077),
 ('?', 35978),
 ('from', 35867),
 ('out', 33995),
 ('up', 30944),
 ('they', 30870),
 ('what', 30791),
 ('is', 30156),
 ('there', 30104),
 ('could', 30078),
 ('were', 29676),
 ('be', 29420),
 ('we', 28147),
 ('not', 28115),
 ('all', 26233),
 ('just', 25647),
 ('him', 25273),
 ('did', 25220),
 ('have', 25073),
 ('would', 24558),
 ('like', 24435),
 ('one', 24288),
 ('into', 24064),
 ('so', 23357),
 ('do', 23220),
 ('back', 22165),
 ('no', 21870),
 ('-', 21187),
 ('when', 21168),
 ('if', 20252),

In [11]:
dataset = [max(stoi2.get(token,stoi2["_unk_"]),0) for token in dataset]

In [12]:
embedding_size = 400

vocab_to_add = new_vocab_size-original_vocab_size

embedder_weights = torch.cat([weights['0.encoder.weight'],torch.zeros(vocab_to_add,embedding_size)])
decoder_weights = torch.cat([weights['1.decoder.weight'],torch.zeros(vocab_to_add,embedding_size)])
new_vocab_size,embedder_weights,decoder_weights

(257874, tensor([[-0.1227,  0.2789, -0.3885,  ..., -0.1040,  0.0196,  0.1855],
         [ 0.0000, -0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [ 0.1807,  1.5874, -0.1174,  ..., -0.0459, -0.0814,  0.1805],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]), tensor([[-0.1227,  0.2789, -0.3885,  ..., -0.1040,  0.0196,  0.1855],
         [ 0.0000, -0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [ 0.1807,  1.5874, -0.1174,  ..., -0.0459, -0.0814,  0.1805],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]))

In [13]:
embedder= nn.Embedding(new_vocab_size,embedding_size,_weight=embedder_weights)
decoder = nn.Linear(new_vocab_size,embedding_size,bias=False)
decoder.weight.data = decoder_weights

In [14]:
rnn0 = nn.LSTM(400, 1150, 1)
input = torch.randn(10, 3, 400)
output0, hn = rnn0(input)

rnn1 = nn.LSTM(1150, 1150, 1)
output1, hn2 = rnn1(output0)

rnn2 = nn.LSTM(1150, 400, 1)
output2, hn2 = rnn2(output1)

rnns = nn.ModuleList([rnn0,rnn1,rnn2])

pprint([(weight_key, weights[weight_key].size()) for weight_key in weights.keys()])
dict((key,value.size()) for key,value in rnns.state_dict().items())

[('0.encoder.weight', torch.Size([238462, 400])),
 ('0.encoder_with_dropout.embed.weight', torch.Size([238462, 400])),
 ('0.rnns.0.module.weight_ih_l0', torch.Size([4600, 400])),
 ('0.rnns.0.module.bias_ih_l0', torch.Size([4600])),
 ('0.rnns.0.module.bias_hh_l0', torch.Size([4600])),
 ('0.rnns.0.module.weight_hh_l0_raw', torch.Size([4600, 1150])),
 ('0.rnns.1.module.weight_ih_l0', torch.Size([4600, 1150])),
 ('0.rnns.1.module.bias_ih_l0', torch.Size([4600])),
 ('0.rnns.1.module.bias_hh_l0', torch.Size([4600])),
 ('0.rnns.1.module.weight_hh_l0_raw', torch.Size([4600, 1150])),
 ('0.rnns.2.module.weight_ih_l0', torch.Size([1600, 1150])),
 ('0.rnns.2.module.bias_ih_l0', torch.Size([1600])),
 ('0.rnns.2.module.bias_hh_l0', torch.Size([1600])),
 ('0.rnns.2.module.weight_hh_l0_raw', torch.Size([1600, 400])),
 ('1.decoder.weight', torch.Size([238462, 400]))]


{'0.weight_ih_l0': torch.Size([4600, 400]),
 '0.weight_hh_l0': torch.Size([4600, 1150]),
 '0.bias_ih_l0': torch.Size([4600]),
 '0.bias_hh_l0': torch.Size([4600]),
 '1.weight_ih_l0': torch.Size([4600, 1150]),
 '1.weight_hh_l0': torch.Size([4600, 1150]),
 '1.bias_ih_l0': torch.Size([4600]),
 '1.bias_hh_l0': torch.Size([4600]),
 '2.weight_ih_l0': torch.Size([1600, 1150]),
 '2.weight_hh_l0': torch.Size([1600, 400]),
 '2.bias_ih_l0': torch.Size([1600]),
 '2.bias_hh_l0': torch.Size([1600])}

In [15]:
rnns.load_state_dict(recast_weights)

In [16]:
from torch import nn

class LangModel(nn.Module):
    
    def __init__(self, embedder, rnns, decoder):
        super(LangModel, self).__init__()
        self.embedder = embedder
        self.rnns = rnns
        self.decoder = decoder
        
    def forward(self,input):
        out=embedder(input)
        for rnn in rnns:
            out,hid = rnn(out)
        out = decoder(out[:,-1])
        return out

In [17]:
import numpy as np
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))
def generate(length,creativity,dist=False):
    next_word = "."
    for i in range (length):
        tensor_output = model(torch.tensor([[stoi2[next_word]]],dtype=torch.long,device="cuda"))[0]
        output = (tensor_output).detach().cpu().numpy()
        
#hard cutoff
#         subdist = softmax(np.sort(output)[-creativity:])
#         print (subdist)
#         next_word = itos2[np.random.choice(np.argsort(output)[-creativity:],p=subdist)]

#soft cutoff
        distribution = softmax(output/creativity)
        ranks = np.argsort(distribution)
        if dist:
            print([itos2[rank] for rank in ranks[-10:]])
            distribution.sort()
            print (distribution[-10:])
            break
        next_word = itos2[np.random.choice(range(distribution.shape[0]),p=distribution)]
        print (next_word,end=" ")
        

In [18]:
model = LangModel(embedder,rnns,decoder)
model.cuda()

LangModel(
  (embedder): Embedding(257874, 400)
  (rnns): ModuleList(
    (0): LSTM(400, 1150)
    (1): LSTM(1150, 1150)
    (2): LSTM(1150, 400)
  )
  (decoder): Linear(in_features=257874, out_features=400, bias=False)
)

In [19]:
generate(50,0.75,False)

the first - flocculation , and the first - kåre , and the first - the previous year - varlyn . the " 2044 , and the " the " the most of the main base , and the best - school of the first - in the " the 

In [20]:
from torch.utils.data import Dataset
import numpy as np
import itertools

class LMDataset(Dataset):
    def __init__(self,text,bptt):
        self.bptt = bptt
        self.text = np.asarray(text)
        
    def __getitem__(self,id):
        dat = np.asarray(self.text[id:id+self.bptt+1])
        if(dat.shape[0] != self.bptt+1):
            print ("SHAPE WRONG! ",dat.shape[0],id)
        result = {
            "obs":dat[:-1],
            "target":dat[-1]
        }

        return result
    
    def __len__(self):
        return len(self.text)-self.bptt

lmdata = LMDataset(dataset,25)

In [21]:
generate(5,0.5,True)

['"', 'he', 'on', 'a', 'this', 'it', 'in', '\n \n ', 'the', '\n ']
[0.00146681 0.00197906 0.00215294 0.00457806 0.00572678 0.00620945
 0.05379397 0.07967424 0.3414945  0.49238464]


In [22]:
from torch.utils.data import DataLoader

trainloader = DataLoader(lmdata,10,shuffle=True,num_workers=1,pin_memory=True)

In [37]:
import torch.optim as optim
from tqdm import tqdm
from time import time
from datetime import timedelta

criterion = nn.CrossEntropyLoss()
lr = 2e-6
decay = 0.0068 #this roughly leads to the rate being halved every 100 times it is applied
# optimizer = optim.Adam(model.parameters(), lr=lr)
creativity = 0.8
checkin_rate = 1000

generate(100,creativity)
print()

start_time = time()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    i=0
    for batch in trainloader:
        inputs = batch["obs"].cuda()
        labels = batch["target"].cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % checkin_rate == checkin_rate-1:    # print every checkin_rate mini-batches
            print('##[{}, {:.2%}] loss: {:.3} lr:{:e} time:{}##'.format(
                  epoch + 1, (float(i) / len(trainloader)), running_loss / checkin_rate,
                lr,
                timedelta(seconds = time()-start_time)))
            
            lr = lr *(1-decay)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

            torch.save(model.state_dict(), "creepy_pasta.weights")
            
            running_loss = 0.0
            generate(100,creativity)
            print()
        
        i+=1

print('Finished Training')

" he ’s not the floor , i heard a few years . " you just very happened . the floor . the kitchen and my world and we were kind of the long also a deep some job or his face . he knew that all of the area by the ground , but i want to myself . the only less in the repeat . i was already sense of what was n’t looked like it . i might take him now to keep my lap . " what it was dragged now . it . if i 
##[1, 0.11%] loss: 5.61 lr:2.000000e-06 time:0:01:06.858196##
he get back into the here back on the point has something you can dream , and we all . she said , " " i smiled . you notice in the room , the bed . there was over to be really was not , i do n’t think i said " i did n’t just i ’ll get up is in the surface . he else as a large enormous ubs to be really considered " i had n’t have gone , an hour , if i was moving . it ’s warning . so i was just 
##[1, 0.23%] loss: 5.58 lr:1.986400e-06 time:0:02:17.990227##
i am clear know it was somehow . my legs . i do so i saw this of the front of

" i have been closed to be in them was true she told me ! " " i was nowhere , " " " this . it with the actual noises and a complete . he killed me . " you wo n’t get the office , and as it was driving out in cvetković . i also to me . i was lost me , around to him . i could not the second . i think i needed to you ’re nearly visible to the air , an 7 a strange in some other noise , more long 
##[1, 2.03%] loss: 5.63 lr:1.780960e-06 time:0:21:17.585721##
" " i could n’t always one of the kitchen . you say to the road with the car . " each only that day away . " walking off about . i knew so wrong , i was going to see a bad when i thought would have seen . that , i understand that languedoc . the alright , i saw something about an hour ? " she was rather , i had been being such an notification , " what , and pulled with its phone a bit of her in the room , the worst . 
##[1, 2.14%] loss: 5.61 lr:1.768849e-06 time:0:22:28.724201##
the hand over , you think why is now ? " " " i could never

they see a scream . they realized that would have to be the car . it ’s been surprised that they have to make any more keeps a poor than her on the well . they know what i have to the one . " alex , the room . we kept across the end , i could hear the darkness now . she could have been gone . he looked forward , but i got a dog . i had been afraid actually remembered the building , i have to play . the detective no black the stairs . 
##[1, 3.95%] loss: 5.6 lr:1.585909e-06 time:0:41:26.372455##
if you . i woke up back the final forehead , seeing exactly to clear quite . " i had been last months and i only a breath . " i know what they ’ve ever played in my eye . " i ’m watching them , i ’ve seen a little age of the look with a tree had to be seen him , and go . i was a few moments . you know , each got to sit - sound for us . " mr. athelstan , and played to be an final island white . " 
##[1, 4.06%] loss: 5.62 lr:1.575124e-06 time:0:42:37.537647##
he then i ’ve given a couple more . th

i ran through the staying on what we were actually look , but i have been a week , i was i am " the wall , and in the best with an empty . " what he had been afraid , with that ’s probably in the blackness . " as a woman , we ’ve entered ( a lot of a creature and josh . " i ’m a few sun , and that my father . i knew that time to his face of the forest . we will not knowing , that ’s like a dry 
##[1, 5.87%] loss: 5.54 lr:1.412220e-06 time:1:01:35.542738##
in a second . " he knows this . " i began to call . i ’ll see her . his breath . you , and upon the corner reading time : " i was n’t even though , and the last . maybe the hands . the walls . " i could n’t quite so much . the whole years , this is no longer strange and my first that said , " asked . i had always been you need to begin ? " " is ’s not a round . i was n’t move of any i 
##[1, 5.98%] loss: 5.59 lr:1.402617e-06 time:1:02:46.516020##
a full of a lot , where it ’s voice lay like love , the salt . in his stuff ; now were us

" hello . he fell into a seat of the end of the kitchen . i ’m not to get me … had kevin . " we ’re a bit were n’t help me , i must have been sat in thud a nice and he pubertal where the slight followed it is another . " it knew it ’s health gone . the mr was the nearest eyes two up all the the words up the candle and ran down on the hours , and his face to the sounds with his phone , a bit of having left 
##[1, 7.78%] loss: 5.53 lr:1.257553e-06 time:1:21:44.037923##
that they worked to be insects . perhaps who continued to double in . however was all the praying ? " get up a smell . so they did n’t make me for what happened to go home . it always stood shocked . " she was there was a few seconds as can come on the road from four . i added a best like , the hell and a lot of the 5-lobed . inside . i was a flashlight and i sat to be a group on the nightmare the good of them and years . " 
##[1, 7.90%] loss: 5.62 lr:1.249002e-06 time:1:22:55.150229##
this point their legs are you look . 

...

even at my life . it . that the tape , but that i my skin , and some exactly if it ’s about her . " she really shifted . this is gone . i wondered ? " " " we ’ve always walked at the window . i do n’t sure he make up for any longer reflection . i did n’t think i ca n’t come . " well i ’ve heard do n’t even did he was n’t be able to find a glance out , a small attic , i was no control . so 
##[2, 90.93%] loss: 5.58 lr:1.950330e-11 time:1 day, 9:30:47.971970##
there ’s what ’s house . i were to she might be a large smile . it did n’t find that a group . out the stories , i ’m sorry and it was on top of the apartment , and i said , i was not to see the door , but in the movement i do n’t want to leave in my stomach . at my head down the nearest and i realized what the blank of the house , i they ’d last years . i ’m not figure looked over the creature , but i woke 
##[2, 91.04%] loss: 5.61 lr:1.937068e-11 time:1 day, 9:31:59.659967##
as i was asleep , she has a little and a young room , b

i think . there was my ears . my business and the tree back together . " oh . i helped it was there was a chance any longer of the bus actually need they were long , her daughter , i had happened to a book , i would have a faint stayed there ’s old , driving . " what i need that they had a he stared at first time . she seemed to be , i ’d been shot and the window . i have been only a chill , she mentioned spirit of but we 
##[2, 92.85%] loss: 5.65 lr:1.736730e-11 time:1 day, 9:51:07.847676##
i ’d try . he said , yes knows what was wrong with stone base of the second he was like one then in empty quickly . like it was known . i ran from the other . it ’d n’t very first moment . so many as much in a forests chrystee , i could n’t want to be three days we were one . soon be a small was she had to the room , whatever to me . the under a real . he retort at least and end . the lights ago , " " then 
##[2, 92.96%] loss: 5.59 lr:1.724920e-11 time:1 day, 9:52:19.501305##
i should have to him ,

i have a moment , i did the night to me , he came a always to his mother , i found you think i will definitely figure the kitchen , i wonder how deep in one - jumped back into my grandfather , though i could n’t ever spent this cause the crowd of his close to the best to where i passed from the screen , sometimes . " what the window , trying to be honest , " hey ’s doubt ’s going with someone . my arms of " there were still as to this one 
##[2, 94.65%] loss: 5.6 lr:1.557111e-11 time:1 day, 10:10:15.327953##
john , so many months and while he given the center of all a moment but i turned out the good . the flesh vanished , which your eyes , but it into the real to try to the boy . " " " hello , however . we have seen - strange . he have to take anyone . the light and over the last end a roof and he looked at first years " . " 12-u_n as we ’re something to be able to my heart yet , " " she would be a person . i 
##[2, 94.76%] loss: 5.63 lr:1.546523e-11 time:1 day, 10:11:27.062171##
ther

" really this whole company an luck ? " i asked my attention , i saw you . like this was they knew of it would believe it ’s mad , i will be the door , as i was still switch , what i had spent the son , but she time . i do n’t put the probably , support . they were sent up , staring at the windows , some next to the wind for when i thought this way , with each looks not below , i followed you were a lane . i could 
##[2, 96.46%] loss: 5.56 lr:1.396069e-11 time:1 day, 10:29:22.529116##
" the rest through a third fine out of the camera . there were running , it was an spirit of him , he he keep the way in , i guess , i ’m sorry to avoid top of my parents . she ’s face . i know what you may be down . she put us as i should n’t here . the ground , and backwards , but found going out . the hallway seemed like a while , " " " remember a few hundred late and there was n’t want to see the edge of 
##[2, 96.57%] loss: 5.53 lr:1.386576e-11 time:1 day, 10:30:34.142971##
two kid . he was my tried 

i ’m sorry . i decided to to be able to hold of us lay for some sound of the cat had been . i ’m sure . she is the shadows " " we are the floor , and we ’re just directed ? " he tried to speak . the road was going these glass and you . i ’d probably quickly of fear . have been far < _unk_ morning , i ’m in my hands that his voice , and the internet , and that i ’m sure . i took us , i was the 
##[2, 98.26%] loss: 5.58 lr:1.251683e-11 time:1 day, 10:48:30.012496##
i got a good , of the same his life had been able to the others and slammed at this is the homologation , and then of air . i had been not even now you will call it was going to the window , ready some of the video , i had been some who will be created of the hand that ’s similar , staring out of your brother to fill under the center of my water and suddenly , and looked like a look after we ’ve been told his stomach , and her chair it all 
##[2, 98.37%] loss: 5.63 lr:1.243171e-11 time:1 day, 10:49:41.776891##
" jonathan rid o

In [28]:
generate(1000,0.75)

i quickly does n’t just barely decided to see if i could you may be sure , like i was a house , no way as he wanted to find this night ! " basement . i could i do n’t want to gather to be released down the shadows . " we might let me to convince him to the other than a dream . " i felt so ever heard the house and i would n’t worry of its body was a large kids has just heard too he did n’t want to have been night and then , i ’m not know ? " how i ca n’t be going on the old and that i ’ll use your shoulders and found my own arms , that , but there ’s throat . the door . the hell was something to my face ’s face . my hands . i heard a huge any people you can only , it was a small . when i ca n’t know you leave . " he did nothing more i got sure that one guest or headed those kid , or her city . " he should ’ve been able to stop . or a long . its surface . i was " was n’t know what they end of a few minutes for a second of the room . " what was completely depression . i did n’t want to hi

In [36]:
import pickle
with open("itos.pkl",'wb') as f:
    pickle.dump(itos2,f)
with open("stoi.pkl",'wb') as f:
    pickle.dump(dict(stoi2),f)
torch.save(model.state_dict(), "creepy_pasta.weights")